In [ ]:
"""
NICOLE JOSEPH
Deep Learning HW 4
Classify CIFAR100
"""

In [3]:
import gzip
import pickle
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import Tensor
from tensorflow import keras
from keras.layers import Conv2D, MaxPooling2D, Add, Flatten, Dropout, ReLU, BatchNormalization, Input, Dense, ZeroPadding2D, AveragePooling2D
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [5]:
# Load the data, similar to how I did for cifar10
# citation: https://www.cs.toronto.edu/~kriz/cifar.html 
def unpickle(file_path):
    data = []
    labels = []
    print("processing " + file_path)
    with open(file_path, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    data = np.append(data, dict[b'data']).astype(np.uint8)
    data = data.reshape(int(len(data)/3072), 3, 32, 32)
    labels = np.append(labels, dict[b'fine_labels']).astype(np.uint8)
    data = data.transpose(0,2,3,1)
    return data, labels

In [6]:
training_dataset = "drive/MyDrive/deep-learning/cifar100/train"
test_dataset = "drive/MyDrive/deep-learning/cifar100/test"

x_train, y_train = unpickle(training_dataset)
x_test, y_test = unpickle(test_dataset)
print("All done!")

processing drive/MyDrive/deep-learning/cifar100/train
processing drive/MyDrive/deep-learning/cifar100/test


In [7]:
training_dataset = "drive/MyDrive/deep-learning/cifar100/train"

drive/MyDrive/deep-learning/cifar100/train


In [8]:
# helper function that takes a tensor as input and adds relu and batch normalization to it
# citation: https://towardsdatascience.com/building-a-resnet-in-keras-e8f1322a49ba 
def relu_bn(inputs: Tensor) -> Tensor:
    relu = ReLU()(inputs)
    bn = BatchNormalization()(relu)
    return bn

In [9]:
# construct a residual block
# citation: https://towardsdatascience.com/building-a-resnet-in-keras-e8f1322a49ba 
def residual_block(x: Tensor, downsample: bool, filters: int, kernel_size: int = 3) -> Tensor:
    y = Conv2D(kernel_size=kernel_size,
               strides= (1 if not downsample else 2),
               filters=filters,
               padding="same")(x)
    y = relu_bn(y)
    y = Conv2D(kernel_size=kernel_size,
               strides=1,
               filters=filters,
               padding="same")(y)

    if downsample:
        x = Conv2D(kernel_size=1,
                   strides=2,
                   filters=filters,
                   padding="same")(x)
    out = Add()([x, y])
    out = relu_bn(out)
    return out

In [10]:
# citation: https://towardsdatascience.com/building-a-resnet-in-keras-e8f1322a49ba 
def create_res_net():
    inputs = Input(shape=(32, 32, 3))
    num_filters = 64
    
    t = BatchNormalization()(inputs)
    t = Conv2D(kernel_size=3,
               strides=1,
               filters=num_filters,
               padding="same")(t)
    t = relu_bn(t)
    
    num_blocks_list = [2, 2, 2, 2]
    for i in range(len(num_blocks_list)):
        num_blocks = num_blocks_list[i]
        for j in range(num_blocks):
            t = residual_block(t, downsample=(j==0 and i!=0), filters=num_filters)
        num_filters *= 2
    
    t = AveragePooling2D(4)(t)
    t = Flatten()(t)
    outputs = Dense(100, activation='softmax')(t)
    
    model = Model(inputs, outputs)

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [11]:
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
# This source has an example of implementing real time data augmentation with the cifar dataset
datagen = ImageDataGenerator(featurewise_center=True,          
                  featurewise_std_normalization=True, 
                  rotation_range=20,
                  width_shift_range=0.1, 
                  height_shift_range=0.1,
                  shear_range=0.05,
                  zoom_range=0.05,
                  horizontal_flip=True,
                  vertical_flip = False)


In [12]:
# citation: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator 
train_generator = datagen.flow(x_train, y_train, batch_size=128)

In [13]:
# citation: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
x_valid = x_train[:100*128]
y_valid = y_train[:100*128]
valid_steps = x_valid.shape[0] // 128
validation_generator = datagen.flow(x_valid, y_valid, batch_size=128)

In [14]:
model = create_res_net()
#model.summary()

In [15]:
#from keras.callbacks import EarlyStopping
#callbacks = [EarlyStopping(monitor='val_loss', patience=1, mode='min', verbose=1)]

In [ ]:
#history = model.fit(train_generator, batch_size=128, epochs=15, verbose=1)

history = model.fit_generator(
                train_generator, 
                steps_per_epoch=len(x_train) // 128,
                epochs=50,
                verbose=1,
                validation_data=validation_generator,
                validation_freq=1,
                validation_steps=valid_steps)
print ("All done!")

In [ ]:
score = model.evaluate(x_test, y_test, verbose=2)
print('Test loss:', score[0])
print('Test accuracy:', score[1])